In [85]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, FloatType
from pyspark.sql.functions import col, sum, count
import datetime


In [86]:
spark = SparkSession.builder.appName('SparkByExamples').getOrCreate()

In [87]:
simpleData = [("account1","2020-01-01",10000), \
    ("account1","2020-02-01",10000), \
    ("account1","2020-03-01",10000), \
    ("account1","2020-04-01",10000), \
    ("account1","2020-05-01",10000), \
    ("account1","2020-06-01",10000), \
    ("account1","2020-07-01",10000), \
    ("account1","2020-08-01",10000), \
    ("account1","2020-09-01",10000), \
    ("account1","2020-09-05",10000), \
    ("account2","2020-10-01",10000), \
    ("account1","2020-10-06",10000), \
    ("account1","2020-11-01",10000), \
    ("account1","2020-11-03",10000), \
    ("account3","2020-12-01",10000), \
    ("account1","2020-12-04",10000), \
    ("account1","2020-05-01",10000), \
    ("account1","2021-01-01",10000), \
    ("account1","2021-01-05",10000), \
    ("account1","2021-02-01",10000), \
    ("account1","2021-05-05",10000), \
    ("account1","01-06-2020",10000), \
    ("account1","01-08-2020",10000), \
    ("account1","01-10-2020",10000), \
    ("account2","01-12-2020",10000), \
    ("account2","01-01-2021",10000), \
    ("account1","01-02-2021",10000), \
    ("account2","01-03-2021",10000), \
    ("account1","2021-03-04",10000), \
    ("account3","2021-03-05",10000) \
  ]

columns= ["account","date","transaction"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.collect()

root
 |-- account: string (nullable = true)
 |-- date: string (nullable = true)
 |-- transaction: long (nullable = true)



[Row(account='account1', date='2020-01-01', transaction=10000),
 Row(account='account1', date='2020-02-01', transaction=10000),
 Row(account='account1', date='2020-03-01', transaction=10000),
 Row(account='account1', date='2020-04-01', transaction=10000),
 Row(account='account1', date='2020-05-01', transaction=10000),
 Row(account='account1', date='2020-06-01', transaction=10000),
 Row(account='account1', date='2020-07-01', transaction=10000),
 Row(account='account1', date='2020-08-01', transaction=10000),
 Row(account='account1', date='2020-09-01', transaction=10000),
 Row(account='account1', date='2020-09-05', transaction=10000),
 Row(account='account2', date='2020-10-01', transaction=10000),
 Row(account='account1', date='2020-10-06', transaction=10000),
 Row(account='account1', date='2020-11-01', transaction=10000),
 Row(account='account1', date='2020-11-03', transaction=10000),
 Row(account='account3', date='2020-12-01', transaction=10000),
 Row(account='account1', date='2020-12-0

In [88]:
df = df.withColumn("date", col("date").cast(DateType())).withColumn("transaction", col("transaction").cast(FloatType()))
df.printSchema()

root
 |-- account: string (nullable = true)
 |-- date: date (nullable = true)
 |-- transaction: float (nullable = true)



In [89]:
df.show()

+--------+----------+-----------+
| account|      date|transaction|
+--------+----------+-----------+
|account1|2020-01-01|    10000.0|
|account1|2020-02-01|    10000.0|
|account1|2020-03-01|    10000.0|
|account1|2020-04-01|    10000.0|
|account1|2020-05-01|    10000.0|
|account1|2020-06-01|    10000.0|
|account1|2020-07-01|    10000.0|
|account1|2020-08-01|    10000.0|
|account1|2020-09-01|    10000.0|
|account1|2020-09-05|    10000.0|
|account2|2020-10-01|    10000.0|
|account1|2020-10-06|    10000.0|
|account1|2020-11-01|    10000.0|
|account1|2020-11-03|    10000.0|
|account3|2020-12-01|    10000.0|
|account1|2020-12-04|    10000.0|
|account1|2020-05-01|    10000.0|
|account1|2021-01-01|    10000.0|
|account1|2021-01-05|    10000.0|
|account1|2021-02-01|    10000.0|
+--------+----------+-----------+
only showing top 20 rows



In [90]:
from datetime import date
now = date.today()
str(now)

'2021-03-20'

In [98]:
from_date = now - datetime.timedelta(days = 90)
from_date1 = now - datetime.timedelta(days = 120)


In [92]:
df.filter(col("date") > from_date).show()

+--------+----------+-----------+
| account|      date|transaction|
+--------+----------+-----------+
|account1|2021-01-01|    10000.0|
|account1|2021-01-05|    10000.0|
|account1|2021-02-01|    10000.0|
|account1|2021-05-05|    10000.0|
|account1|2021-03-04|    10000.0|
|account3|2021-03-05|    10000.0|
+--------+----------+-----------+



In [104]:
df1 = df.filter(col("date") > from_date).groupby("account").agg(sum("transaction").alias("sum_transactions_last3m"))
df1.show()

+--------+-----------------------+
| account|sum_transactions_last3m|
+--------+-----------------------+
|account1|                50000.0|
|account3|                10000.0|
+--------+-----------------------+



In [105]:
df2 = df.filter(col("date") > from_date1).groupby("account").agg(sum("transaction").alias("sum_transactions_last6m"))
df2.show()

+--------+-----------------------+
| account|sum_transactions_last6m|
+--------+-----------------------+
|account1|                60000.0|
|account3|                20000.0|
+--------+-----------------------+



In [106]:
df3 = df1.join(df2, on = "account", how = "inner")
df3.show()

+--------+-----------------------+-----------------------+
| account|sum_transactions_last3m|sum_transactions_last6m|
+--------+-----------------------+-----------------------+
|account1|                50000.0|                60000.0|
|account3|                10000.0|                20000.0|
+--------+-----------------------+-----------------------+

